In [1]:
##################################
#####-------NOTATKI---------######
##################################

#LEPSZA METODA WYPEŁNIENIA KONTURU
#ZNALEŚĆ SPOSÓB NA POZBYCIE SIĘ ZIARNA
#EWENTUALNE WYOSTRZANIE

#BIG BOSS
#AUTOMATYZACJA DOBORU PARAMETRÓW


#####################################
#####-------BIBLIOTEKI---------######
#####################################

import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

####################################
#####-------PARAMETRY---------######
####################################

ksize_inverse = 3
ksize_normal = 3

area_size_inverse = (1,1)
area_size_normal = (1,1)
threshold_for_mask_inverse = 0.75
threshold_for_mask_normal = 0.2

threshold_inverse = 70
threshold_normal = 180

kernel_size_close = (5,5)

kernel_size_dilate = (5,5)

TELEA_size = 5

print_mid_result = 0

load="dotestu/baba4.jpg"
save="wyniki/result.jpg"

#################################
#####-------BACKUP---------######
#################################
#ksize_inverse = 3
#ksize_normal = 3
#area_size_inverse = (1,1)
#area_size_normal = (1,1)
#threshold_for_mask_inverse = 0.75
#threshold_for_mask_normal = 0.2
#threshold_inverse = 70
#threshold_normal = 180
#kernel_size_close = (5,5)
#kernel_size_dilate = (5,5)
#TELEA_size = 5
#print_mid_result = 0
#load="dotestu/baba3.jpg"
#save="wyniki/result.jpg"

In [2]:
def show_image(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def get_mask(magnitude, magnitude_threshold, area_size):
    # Threshold the gradient magnitude image
    magnitude_black_white = magnitude > magnitude_threshold * np.max(magnitude)
    
    # Perform morphological operations to remove small objects and fill holes
    magnitude_black_white = cv2.morphologyEx(magnitude_black_white.astype(np.uint8), cv2.MORPH_CLOSE,
                            cv2.getStructuringElement(cv2.MORPH_ELLIPSE, area_size))
    return magnitude_black_white

In [4]:
# Load the input image
image = cv2.imread(load)
image_work = image.copy()
image_work_inverse = image.copy()
image_work_normal = image.copy()

In [5]:
# Inverse by subtracting from 255
image_work_inverse = 255 - image

if print_mid_result == 1:
    show_image('input', image)
    show_image('inverse', image_work_inverse)

In [6]:
# Convert image to grayscale
gray_image_inverse = cv2.cvtColor(image_work_inverse, cv2.COLOR_BGR2GRAY)
gray_image_normal = cv2.cvtColor(image_work_normal, cv2.COLOR_BGR2GRAY)

if print_mid_result == 1:
    show_image('gray_image_inverse', gray_image_inverse)
    show_image('gray_image_normal', gray_image_normal)

In [7]:
# Compute the image gradients using Sobel operators
gx = cv2.Sobel(gray_image_inverse, cv2.CV_64F, 1, 0, ksize_inverse)
gy = cv2.Sobel(gray_image_inverse, cv2.CV_64F, 0, 1, ksize_inverse)

#NOWA WERSJA
abs_grad_x = cv2.convertScaleAbs(gx)
abs_grad_y = cv2.convertScaleAbs(gy)
mg_inverse = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

# Compute the image gradients using Sobel operators
gx = cv2.Sobel(gray_image_normal, cv2.CV_64F, 1, 0, ksize_normal)
gy = cv2.Sobel(gray_image_normal, cv2.CV_64F, 0, 1, ksize_normal)

#NOWA WERSJA
abs_grad_x = cv2.convertScaleAbs(gx)
abs_grad_y = cv2.convertScaleAbs(gy)
mg_normal = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

In [8]:
# Threshold the gradient magnitude image
mask_inverse = get_mask(mg_inverse, threshold_for_mask_inverse, area_size_inverse)
mask_normal = get_mask(mg_normal, threshold_for_mask_normal, area_size_normal)

if print_mid_result == 1:
    show_image('firstmask_inverse', mask_inverse*255)
    show_image('firstmask_normal', mask_normal*255)

In [9]:
# Substitude binary true pixels with colors from original image using cv2
mask_inverse = cv2.bitwise_and(image_work_inverse, image_work_inverse, mask=mask_inverse.astype(np.uint8))
mask_normal = cv2.bitwise_and(image_work_normal, image_work_normal, mask=mask_normal.astype(np.uint8))

# convert to grayscale    
mask_inverse = cv2.cvtColor(mask_inverse, cv2.COLOR_BGR2GRAY)
mask_normal = cv2.cvtColor(mask_normal, cv2.COLOR_BGR2GRAY)

if print_mid_result == 1:
    show_image('bitwise_inverse', mask_inverse)
    show_image('bitwise_normal', mask_normal)

In [10]:
# remove all pixels with value below threshold using opencv
mask_inverse = cv2.threshold(mask_inverse, threshold_inverse, 255, cv2.THRESH_BINARY)[1]
mask_normal = cv2.threshold(mask_normal, threshold_normal, 255, cv2.THRESH_BINARY)[1]

if print_mid_result == 1:
    show_image('threshold_inverse', mask_inverse)
    show_image('threshold_normal', mask_normal)

#add two mask
mask_add = cv2.add(mask_inverse,mask_normal)

if print_mid_result == 1:
    show_image('add', mask_add)

#domykanie konturu WERSJA BETA
M=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size_close)
mask_add=cv2.morphologyEx(mask_add, cv2.MORPH_CLOSE, M)

if print_mid_result == 1:
    show_image('close mask', mask_add)
    
# Save mask
cv2.imwrite('wyniki/mask.jpg', mask_add)
cv2.imwrite('wyniki/mask_inverse.jpg', mask_inverse)
cv2.imwrite('wyniki/mask_normal.jpg', mask_normal)

True

In [11]:
#draw mask on the image
contours, hierarchy = cv2.findContours(mask_add, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image_work, contours, -1, (0, 0, 255), 1)
if print_mid_result == 1:
    show_image('contour', image_work)
#save contour
cv2.imwrite('wyniki/contour.jpg', image_work)

True

In [12]:
#apply morphology dilate
wide_mask = mask_add.copy()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size_dilate)
wide_mask = cv2.morphologyEx(wide_mask, cv2.MORPH_DILATE, kernel)
if print_mid_result == 1:
    show_image('contour', wide_mask)
cv2.imwrite('wyniki/wide_mask.jpg', wide_mask)

True

In [13]:
# Paint the mask using nearest neighbor interpolation
final_image = cv2.inpaint(image, wide_mask, TELEA_size, cv2.INPAINT_TELEA)

cv2.imshow('input', image)
cv2.imshow('contour', image_work)
cv2.imshow('output', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite(save, final_image)

True